# Simulating "Angry Birds" type projectile motion and collision
Author: 18003453
Last updated: 3.01.2020

## Introduction

### Aims

In this document a simulator will be built to model the projectile motion of a 'bird', which will be assumed to act as a point like object. This object will start from a randomized height between 0 and 1 units, each unit equals 1 metre, it will then be fired from this point with a specific trajectory relative to the horizontal and a specific velocity at this angle. This motion will then be animated via the use of vPython coding tools before the the 'bird' then either hits the given target, a verticle rectangle, or falls to the ground either side of the target. If this bird does collide with the target, a calculation will then be performed to find out whether the target will be toppled or stay standing. This will be done using physical values for the mass and dimensions of both objects aswell as the velocity and position of the bird at the time of collision.
   
When either the bird collides with the ground or collides with the target but fails to topple it the game will reset to the initial firing stage where the launch angle and velocity are inputted. This will occur until the bird collides with the target has is calculated to topple it. At each stage whether the bird hits the ground or hits the target and it topples or not, labels will pop up on the screen notifying the player of the outcome of the last attempt. The relevant data for each attempt will be outputted below the game in string format, this is useful to the user as the next inputted values can take this data into account.

After completing this base structure for the game, more superficial attributes will be added and altered to allo more enjoyable game-play. This will include the animation of the toppling of the tower and better cartoons for the objects within the game.

### Strategy

The code will be separated into sections and developed individually before then being linked together. This will hopefully allow for increased efficiency in debugging and finalising each of the features of the simulator. The whole code will be brought together using two overarching while loops, one for whether the bird hits the target, if not the game will reset, and the other for when the bird hits the target it topples or not,  if not the game will reset again. While loops will also be used for all animation purposes of the bird flying and the target toppling. 

### Importing Modules

In [1]:
# Importing numpy and vpython module for modelling and animation purposes

import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, arrow, radians, textures, compound, cone, ellipsoid, cylinder


<IPython.core.display.Javascript object>

### Physics Coding

The physics used treats the bird as a point object for all intensive purposes, this includes collision with the target and the ground as well as its projectile motion. 

SUVAT formulae have been used to model the flight path of the bird from the target height all the way to where it collides with an object, the vertical and horizontal positions of the bird were calculated at each incrementing time value and the bird's position was updated. As gravity is a constant downwards acceleration the equation, $y = u_y*t + 0.5*(-g)*t^2$ was used, where $u_y$ is the initial vertical compenent of velocity. Horizontal position is rather simple as the horizontal component of velocity does not change and so the equation $x = u_x * t $ is used, where $u_x$ is the horizontal component of velocity. Assumptions made include the bird acting as a point particle in terms of motion and collisions with the ground and the target, it was also assumed that air resistance will have no effect on the bird's motion. 

As the bird flies through the air, the components of momentum at each point are also calculated using simple SUVAT equations multiplied by the bird's 'mass'. When the collision occurs the while loop stops and the momentum is no longer updated, it stays at the value just before collision. From this point on if the collision is a 'side_collision' the program will then calculate the torque the bird collision applies on the target, using the equation $ T_a = (p_x/T)*y $ where $T_a$ is the applied torque on the target, $p_x$ is the horizontal momentum and T is the contact time, y being the height of contact from the ground. This value is then compared to the restoring torque due to the Targets own mass, Eq. 7 in Final assignemtn script, and if the applied torque is larger then the target will topple. For collision if there bird enters the Target then the code calculates if the bird is in a small rectangle at the front which would give a side_collision and so a topple calculation, otherwise the bird must have landed on top of the target.

### Simluation of Projectile motion


In [2]:
#  Game Setting #############################


# Code adapted from "PHAS0007: Final assignment 2019/20" (Dash, L. 2019)


# plotting constant game objects
scene = canvas(width=640, height=480, center=vector(8,5,0),range=8, background= vector(0.436, 0.792, 0.838)) #sets scene
ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green, radius = 0.26) #plots ground
earth = box(pos = vector(8,-4,0),length=16, height=8, width=0.5, color=vector(1.000, 0.678, 0.207),texture = textures.stucco)
 
    
# randomized variables
xT = 5 + 10*random()
yB = random()

# setting parameter values
g = 9.81     #gravitational acceleration, m/s^2
dt = 0.0001  #time interval for loop animation, in seconds
m = 0.1 #mass of bird
M = 100 #mass of target

# plotting variable/animated objects 
platform = cylinder(pos=vector(0,yB,0), axis=vector(-0.9,-0.7,0), radius=0.35, color=vector(0.000, 0.339, 0.723)) #plots cannon
cannon_sphere = sphere(pos=vector(-0.9,yB-0.7,0), radius = 0.43, color = vector(0.933, 0.261, 0.135)) #plots bottom of cannon
pointer = arrow(pos = vector(0,yB,0),  axis=vector(0,0,0), shaftwidth=0.1, color = color.magenta) #setting values of momentum pointer
target = box(pos=vector(xT,1,0),length=0.5, height=2, width=0.5, texture ="https://i.imgur.com/OvtKXiw.jpg")           #plots target box

# creating bird object at centre of canvas
body = sphere(pos = vector(8,5,0),radius = 0.3, color=vector(0.796, 0.104, 0.104), shininess = 0) #setting values of bird
left_eye = sphere(pos = vector(8.24,5.1,-0.07),radius = 0.07, color=color.white, shininess = 0.2) #setting values of bird
right_eye = sphere(pos = vector(8.24,5.1,0.07),radius = 0.07, color=color.white, shininess = 0.2) #setting values of bird
left_pupil = sphere(pos = vector(8.29,5.1,-0.07),radius = 0.026, color=color.black, shininess = 0.2) #setting values of bird
right_pupil = sphere(pos = vector(8.29,5.1,0.07),radius = 0.026, color=color.black, shininess = 0.2) #setting values of bird
beak = cone(pos = vector(8.25,4.95,0), axis=vector(1,0,0), color=vector(1.000, 0.642, 0.000), size=vector(0.2,0.17,0.3), shininess = 0.1)
left_eyebrow = box(pos = vector(8.27,5.17,-0.07), axis=vector(0,-0.5,1), color=color.black ,size=vector(0.2,0.08,0.04))
right_eyebrow = box(pos = vector(8.27,5.17,0.07), axis=vector(0,-0.5,-1), color=color.black ,size=vector(0.2,0.08,0.04))
chest = ellipsoid(pos=vector(8.15,4.85,0), size=vector(0.4,0.2,0.33), axis=vector(1,0.8,0), color=vector(1.000, 0.888, 0.723)) 
feather_1 = cylinder(pos=vector(8.07,5.05,0), axis=vector(-0.2,0.34,0), radius=0.04, color=vector(0.796, 0.104, 0.104), shininess = 0)
feather_2 = cylinder(pos=vector(8.07,5.05,0), axis=vector(-0.07,0.37,0), radius=0.04, color=vector(0.796, 0.104, 0.104), shininess = 0)
bird = compound([body, left_eye, right_eye, beak, left_eyebrow, right_eyebrow, chest, feather_1, feather_2, left_pupil, right_pupil])  #grouping all objects together
bird.pos = vector(0,yB,0)  #setting position to the platform height


# label plotting
TloseLab = label(pos=vector(8,6,-0.5), text='The Target has not toppled this time', color = color.black, box = False, height = 18, visible = False)  #outputting loss label when collision does not topple target
loseLab = label(pos=vector(8,6,-0.5), text='The Bird has missed the target', color = color.black, box = False, height = 18, visible = False)  #outputting loss label bird misses the target
topLab = label(pos=vector(8,6,-0.5), text='The Bird has hit the top of the target', color = color.black, box = False, height = 18, visible = False)  #outputting loss label bird misses the target
tryAgain = label(pos=vector(8,4.5,-0.5), text='Click anywhere to Try Again', color = color.black, height = 25, visible = False, box = False)  #outputting loss label when collision does not topple target
winLab = label(pos=vector(8,6,-0.5), text='Congratulations! You Win!!!', color = color.black, border = 0, height = 25, visible = False, box = False)  #outputting success label when collision causes target to topple
clickPlay = label(pos=vector(8,6,-0.5), text='Click anywhere to Play', color = color.black, border = 0, height = 25, visible = True, box = False)  #outputting success label when collision causes target to topple




#######################  Progression coding (I) #############################

# intitialising topple and good_contact
topple = False
good_contact = False
n = 1  #attempt counter initialisation, always starts on 1st attempt

# collision function, tests if bird is within target
def collision():
    if y<=2 and (((xT-0.25) <= x <= (xT+0.25))):
        return True
    else:
        return False

# side_collision function, tests if bird hits side of target
def side_collision():
    if x > ((xT-0.25) - 0.01) and x < ((xT-0.25) + 0.01) and y<=2:
        return True
    else:
        return False

# function to hide all labels and wait to to redraw
def hide_labels():
    TloseLab.visible = False #making target non topple label invisible
    loseLab.visible = False #making miss label invisible
    topLab.visible = False #making hitting top label invisible
    tryAgain.visible = False #making Try Again label invisible 
    clickPlay.visible = False  #making Click anywhere to Play label invisible
    scene.waitfor('redraw') #halts code until canvas is updated
    

scene.pause('')    #pausing code until canvas is clicked
print("Aim: Topple the Target")   #prints aim of game
clickPlay.visible = False  #hides label saying 'click to play'


# while loops for different impact scenarios
while topple == False:       #for restarting when bird collides with target but it does not topple
    while good_contact == False:   #for restarting when bird collides with ground
        
        
        bird.pos = vector(0,yB,0)  #resets bird position back to platform 
        scene.waitfor('redraw')  #halts code until canvas is updated  
        
        
        
#######################  Simulation of Projectile Motion #############################


        # Code adapted from my submission for PHAS0007 Session 8

    
        # input initial conditions (angle and speed)
        print()
        print("Attempt", n)   #logging attempt number
        dtheta = float(input("Input the launch angle in degrees from the horizontal: "))
        theta = np.radians(dtheta)   #converting launch angle from degrees to radians, which python understands
        v0 = float(input("Input the initial speed in metres/second (between 0 - 150m/s):"))
        

        # resetting parameter values
        x = 0   #initialising value for bird's x coordinate
        y = yB + 0.3  #resetting bird's y coordinate to just aboce original platform y coordinate 
        t = 0   #initialising time
        Px = 0  #initialising value for Px
        Py = 0  #initialising value for Py
        

        # while loop for animation of bird flight, bird is plotted and calculated as a point particle
        while y >= 0 and collision() == False:   #while bird is above ground and not within the target
            rate(3000)                         #runs animation at 3000 increments per seconds of 0.0001 second increments, so simulates 0.3x real-world timings
            bird.pos = vector(x,y,0)            #redraws bird at current x,y values, code is above x,y equations to prevent bird going below 0
            pointer.pos = vector(x,y,0)         #redraws arrow at current x,y values
            pointer.axis = vector(Px,Py,0)      #redraws arrow axis with current momentum vectors
            bird.axis = vector(Px,Py,0)         #bird rotates with axis of motion
            t = t + dt                          #incrementing t by value dt
            x = v0 * np.cos(theta) * t          #finding value of x with respect to t following classical mechanics equations
            y = yB + (v0 * np.sin(theta) * t) - (0.5 * g * t**2)   #finding value of y with respect to t following classical mechanics equations
            Px = m * v0 * np.cos(theta)         #finding x component of momentum at time, t
            Py = (m * v0 * np.sin(theta)) - (m * g * t)  #finding y component of momentum at time, t
            


        # adjustments for animation end
        pointer.axis = vector(0,0,0)  #setting momentum pointer to zero magnitude as bird has stopped moving
        bird.axis = vector(0,0,0)     #setting bird axis back to horizontal
    
        
#######################  Progression coding (II) #############################        
        
        
        # attempt counter
        n = n + 1 #incrementing attempt number
        
            
        # if statement to determine where bird collision has occurred
        if side_collision() == True:        #if bird is within a thin rectangle area on the front of the target
            momentum = mag(vector(Px,Py,0))  #calculating magnitude of momentum of bird on impact with target
            print('The bird impacted the target', y, 'm from the ground.')    #printing impact height of bird on target
            print('On impact with the target the bird had a momentum of', momentum, 'kg m/s.')  #printing impact momentum of bird
            good_contact = True     #ending good_contact while loop to calculate whether target topples
            scene.waitfor('redraw')  #halts code until canvas is updated
        else: 
            
            if y <=0:  #if bird has hit ground
                print('The bird flew', x, 'm.')     #printing flight distance of bird
                print()
                loseLab.visible = True   #making miss label visible
            else:      #bird must have hit the top of the target
                print('The bird has hit the top of the target.')     #printing bird has hit the top
                print()     #space
                topLab.visible = True   #making hitting top label visible
            tryAgain.visible = True  #making Try Again label visible
            scene.pause('') #pausing code until canvas is clicked
            hide_labels()   #making labels invisible while bird is in flight
             
        # good_contact while loop close
    

    

#######################  Topple Calculation #############################



    # torque calculations
    F_app = Px/0.01  #applied force on target due to bird collision
    T_app = cross(vector(F_app,0,0), vector(-0.5,y,0)) #cross product or applied force vector and collision position vector from tipping point
    T_rest = g * M * 0.25 #value for restoring torque
    
    # outputting torque calculations
    print ("The applied torque due to the bird collision was", mag(T_app), "Nm")     #outputting applied torque value
    print ("The magnitude of the restoring torque of the target was ", T_rest, "Nm") #outputting restoring torque value
    
    
    
#######################  Progression coding (III) ############################# 


    # if statement for outputting whether target will topple
    if mag(T_app) > T_rest: #target does topple
        
        
        # animation for toppling of target and falling of ball
        topple_angle = 1  #setting counting variable
        while topple_angle <= 45:
            rate(60)
            target.rotate(angle= radians(90)/45, axis=vector(0,0,-1), origin=vector(xT+0.25,0,0))  #rotating target around bottom corner
            topple_angle = topple_angle + 1   #incrementing counter variable
            bird.pos = vector(x, (y - y*(topple_angle/45)), 0)  #falling of bird to ground
            scene.waitfor('redraw')  #halts code until canvas is updated
        
        
        winLab.visible = True #making win label visible
        print ("The target has toppled! Congratulations, you have won!!!")  #printing success message
        print()   #space
        topple = True   #ending contact while loop by setting topple to True, as game is complete
        scene.waitfor('redraw')  #halts code until canvas is updated
        
    else:   #target does not topple, resets game and outputs required labels
        TloseLab.visible = True  #making non topple loss label visible
        tryAgain.visible = True  #making Try Again label visible
        print ("The target has not toppled this time, give it another go.")  #printing faliure message
        print()     #space
        good_contact = False    #resetting good_contact to False as attempt has failed so game is reset
        scene.pause('')   #pausing code until canvas is clicked
        hide_labels()   #making labels invisible
        
        
    # topple while loop close

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Aim: Topple the Target

Attempt 1
Input the launch angle in degrees from the horizontal: 15
Input the initial speed in metres/second (between 0 - 150m/s):20
The bird impacted the target 1.7879681273210708 m from the ground.
On impact with the target the bird had a momentum of 1.944967707820652 kg m/s.
The applied torque due to the bird collision was 345.40891815222466 Nm
The magnitude of the restoring torque of the target was  245.25 Nm
The target has toppled! Congratulations, you have won!!!



## Improvements

The bird could be calculated as an object with a size rather than a point like object, air resistance could also be added to the bird to give a better simulation of the real world. For collsions it would be better to include a better sense checker for if the bird hits the side of the target as currently, if the bird has a horizontal velocity larger than 200m/s the program will potentiall not realise that is has collided with the side of the target and will output at 'Top Collision' instead.

###### Bibliography

* Dash, L (2019). _"PHAS0007: Final assignment 2019/20"_. UCL Moodle resource available from https://moodle.ucl.ac.uk/course/view.php?id=10648&section=14 [accessed 16 January 2020\]

* Chabay, Ruth, et al. (2017) “VPython Help.” Glowscript.Org, www.glowscript.org/docs/VPythonDocs/index.html. [accessed 16 January 2020\]